In [8]:
%matplotlib inline

import os
import sys
import time
import ctypes
import multiprocessing
import warnings

import dicom
import csv
import pickle
import scipy
import numpy as np
import pandas as pd
import tensorflow as tf

from copy import deepcopy
from collections import namedtuple
from scipy.misc import imread
from scipy import stats
from scipy import ndimage
from matplotlib import pyplot as plt

In [9]:
# Utility functions

def save(**kwargs):
    with open('save.dat', 'wb') as f:
        pickle.dump(kwargs, f)


def load():
    with open('save.dat', 'rb') as f:
        return pickle.load(f)

## Read Basic Patient Data
* Use '7z x stage1.7z' to uncompress data.

In [20]:
_DATA_DIR = '../ct_data/sample_images'
_LABELS_CSV = '../ct_data/stage1_labels.csv'
_SAMPLE_CSV = '../ct_data/stage1_sample_submission.csv'


def read_patient_labels(csv_fname):
    with open(csv_fname) as f:
        reader = csv.reader(f, delimiter=',')
        reader.next()  # skip header
        return dict([(r[0], float(r[1])) for r in reader if len(r) == 2])

    
def read_patient_data(patient_name):
    d = os.path.join(_DATA_DIR, patient_name)
    return [
        dicom.read_file(os.path.join(d, f))
        for f in os.listdir(d)
    ]

In [11]:
patient_names = os.listdir(_DATA_DIR)
patient_labels = read_patient_labels(_LABELS_CSV)
test_patient_names = set(read_patient_labels(_SAMPLE_CSV).keys())

## Preprocess Data

## Lung Segmentation